In [1]:
from configparser import ConfigParser
import mysql.connector as mysql
import pandas as pd
import math


FIRE_COLS = ['UniqueFireIdentifier', 'IncidentName', 'CreatedOnDateTime_dt', 'FireDiscoveryDateTime',
             'Latitude', 'Longitude', 'InitialLatitude', 'InitialLongitude', 'GACC', 'ContainmentDateTime',
             'FireOutDateTime', 'ControlDateTime', 'DiscoveryAcres', 'IncidentSize', 'FinalAcres', 'InitialResponseAcres']


config = ConfigParser()
config.read('../config.ini')

db = mysql.connect(
    user=config['db']['user'],
    password=config['db']['password'],
    database=config['db']['database'],
    host=config['db']['host']
)
cursor = db.cursor()

cursor.execute('SELECT * FROM FirePoint')
fires = cursor.fetchall()
db.close()

fire_locations = pd.DataFrame(fires, columns=FIRE_COLS)

fire_locations.head()

,UniqueFireIdentifier,IncidentName,CreatedOnDateTime_dt,FireDiscoveryDateTime,Latitude,Longitude,InitialLatitude,InitialLongitude,GACC,ContainmentDateTime,FireOutDateTime,ControlDateTime,DiscoveryAcres,IncidentSize,FinalAcres,InitialResponseAcres
0,2003-WYCMX-200237,SCHWARTZ,2020-07-04 16:03:33,2003-01-01 07:00:00,44.6033399940001,-105.5630669220000,None,None,RMCC,None,None,None,NaN,NaN,NaN,NaN
1,2004-CABDU-006784,MENTONE,2020-10-13 01:01:01,2004-07-06 01:04:00,34.7338938890001,-117.0766787120000,None,None,OSCC,None,None,None,NaN,NaN,NaN,NaN
2,2008-AZPNF-000975,Mingus/Cherry Rx,2019-09-19 21:08:47,2008-10-07 18:16:00,34.5983062720001,-112.0724141090000,34.3899990000000,-112.0800020000000,SWCC,None,None,None,NaN,NaN,NaN,NaN
3,2008-IDCTF-008902,St. Charles RX,2019-08-28 15:23:44,2008-09-08 13:35:00,42.0963362210001,-111.4269118300000,42.1038890000000,-111.5477780000000,GBCC,None,None,None,1.0,20.0,NaN,NaN
4,2008-IDGBK-000002,NIFC RAMP SUPPORT,2019-08-05 14:33:55,2008-07-22 15:10:00,43.6666726250000,-116.2166796910000,None,None,GBCC,None,None,None,NaN,NaN,NaN,NaN


Let's take a look at which attributes are not well tracked (i.e. which ones have null values for lots of instances)

In [2]:
total_instances = fire_locations.shape[0]
print("Total instances: " + str(total_instances))
missing_values = fire_locations.isnull().sum()
print("Missing values for each attribute:")
for attribute, count in missing_values.items():
    print(f"{attribute}: {count} ({int(count/total_instances * 10000)/100}%)")

Total instances: 247742
Missing values for each attribute:
UniqueFireIdentifier: 0 (0.0%)
IncidentName: 0 (0.0%)
CreatedOnDateTime_dt: 0 (0.0%)
FireDiscoveryDateTime: 0 (0.0%)
Latitude: 0 (0.0%)
Longitude: 0 (0.0%)
InitialLatitude: 65980 (26.63%)
InitialLongitude: 65982 (26.63%)
GACC: 59 (0.02%)
ContainmentDateTime: 97409 (39.31%)
FireOutDateTime: 103400 (41.73%)
ControlDateTime: 111877 (45.15%)
DiscoveryAcres: 63894 (25.79%)
IncidentSize: 74333 (30.0%)
FinalAcres: 229703 (92.71%)
InitialResponseAcres: 161222 (65.07%)


Remove instances with missing values for the attributes FireOutDateTime and IncidentSize because we need both for calculations

In [3]:
fire_locations.dropna(subset=["FireOutDateTime"], inplace=True)
fire_locations.dropna(subset=["IncidentSize"], inplace=True)

print("Total instances: " + str(fire_locations.shape[0]))

Total instances: 136992



Remove UniqueFireID _2014-IDNCF-000609_ and _2014-AKFAS-411093_ because they have dates in 1530.

Then, find the difference between each fire's discovery date and time and it's FireOut date and time to get the total time burned. Display the first few rows to verify the calculation. Finaly, get the hours burned from the time burned so that we're working with a consistent time unit.

In [4]:
fire_locations.drop(fire_locations[fire_locations['UniqueFireIdentifier'] == '2014-AKFAS-411093'].index, inplace=True)
fire_locations.drop(fire_locations[fire_locations['UniqueFireIdentifier'] == '2014-IDNCF-000609'].index, inplace=True)

fire_locations['FireDiscoveryDateTime'] = pd.to_datetime(fire_locations['FireDiscoveryDateTime'])
fire_locations['FireOutDateTime'] = pd.to_datetime(fire_locations['FireOutDateTime'])
fire_locations['TimeBurned'] = fire_locations['FireOutDateTime'] - fire_locations['FireDiscoveryDateTime']

print(fire_locations[['FireDiscoveryDateTime', 'FireOutDateTime', 'TimeBurned']].head())

fire_locations['HoursBurned'] = fire_locations['TimeBurned'].dt.total_seconds() / (60*60)

   FireDiscoveryDateTime     FireOutDateTime      TimeBurned
21   2009-07-06 20:00:00 2009-07-07 20:00:00 1 days 00:00:00
75   2014-05-13 21:31:58 2014-05-17 02:00:00 3 days 04:28:02
76   2014-05-25 22:20:45 2014-05-27 03:00:00 1 days 04:39:15
77   2014-05-29 02:06:19 2014-05-29 07:07:00 0 days 05:00:41
78   2014-06-04 00:18:20 2014-06-04 02:23:00 0 days 02:04:40


Now, let's get an idea of what we're working with using the range and average of both attributes we'll be working with for suppression result. We want to get an idea of the distribution of our data.

In [5]:
summary = fire_locations[['HoursBurned', 'IncidentSize']].describe()

time_burned_range = summary.loc['max', 'HoursBurned'] - summary.loc['min', 'HoursBurned']
acres_burned_range = summary.loc['max', 'IncidentSize'] - summary.loc['min', 'IncidentSize']
average_time_burned = summary.loc['mean', 'HoursBurned']
average_acres_burned = summary.loc['mean', 'IncidentSize']

print("HoursBurned Range:", time_burned_range)
print("HoursBurned Average:", average_time_burned)
print("IncidentSize Range:", acres_burned_range)
print("IncidentSize Average:", average_acres_burned)

HoursBurned Range: 28090.043611111112
HoursBurned Average: 296.85857245662703
IncidentSize Range: 589368.0
IncidentSize Average: 428.08532922111095


Quite the range there, and we know from Kole's graphs that especially for acres burned we have a ton of very small fires and not many large ones. With such a dramatic of a right (or positive) skew, we won't be able to fully normallize the data.

![Acres Burned Distribution](./acres-distribution-chart.svg)

We'll use a log to normalize as much as we can. We'll also need to transform our data (x + 1) to make sure we don't get negative values from the fires that burned for less than an acre or less than an hour.

SuppressionResult is calculated using the average of normalized time burned and normalized acreage burned. Acreage burned is given twice as much weight as time burned because we think it is a better indicator of how well a fire was supressed. This average is then converted to a percent.

SuppressionResult = $\left (1 - \frac{\displaystyle\left(\frac{\log(x + 1)}{\log(\text{MaxHoursBurned} + 1)} + \frac{2(\log(y + 1))}{\log(\text{MaxAcresBurned} + 1)}\right)}{\bigg(3\bigg)}\right) \times 100$

Where $x = HoursBurned$ and $y = AcresBurned$ of a given fire

In [6]:
log_time_range = math.log(time_burned_range+1)
log_acres_range = math.log(acres_burned_range+1)
print("log time: " + str(log_time_range) + " log acres: " + str(log_acres_range))

fire_locations["NormalizedTime"] = fire_locations["HoursBurned"].apply(lambda x: math.log(x+1)/log_time_range if x > 0 else 0)
fire_locations["NormalizedAcreage"] = fire_locations["IncidentSize"].apply(lambda x: math.log(x+1)/log_acres_range if x > 0 else 0)
fire_locations['SuppressionResult'] = (1 - (fire_locations['NormalizedTime'] + (2 * fire_locations['NormalizedAcreage']))/3) * 100

log time: 10.243206071815102 log acres: 13.286807752042323


Finally, let's take a look how well our suppression result measures up against both the bigger fires and the rest of the data (mostly much smaller fires).

In [7]:
filtered_fire_locations = fire_locations[fire_locations['IncidentSize'] > 500]
print(filtered_fire_locations[['FireDiscoveryDateTime', 'HoursBurned', 'IncidentSize',
                               'NormalizedTime', 'NormalizedAcreage', 'SuppressionResult']].head())

print(fire_locations[['FireDiscoveryDateTime', 'HoursBurned', 'IncidentSize',
                      'NormalizedTime', 'NormalizedAcreage', 'SuppressionResult']].head())

    FireDiscoveryDateTime  HoursBurned  IncidentSize  NormalizedTime  \
170   2014-07-08 01:29:00   510.516667        1000.0        0.608929   
187   2014-08-12 19:22:00   716.633333        2632.0        0.641982   
189   2014-08-13 05:57:00   162.183333        1200.0        0.497391   
190   2014-08-13 06:20:00   538.666667         560.0        0.614158   
241   2014-05-20 00:03:50  4866.936111      196610.0        0.828884   

     NormalizedAcreage  SuppressionResult  
170           0.519971          45.037649  
187           0.592759          39.083296  
189           0.533680          47.841616  
190           0.476391          47.768625  
241           0.917375          11.212228  
   FireDiscoveryDateTime  HoursBurned  IncidentSize  NormalizedTime  \
21   2009-07-06 20:00:00    24.000000           0.1        0.314245   
75   2014-05-13 21:31:58    76.467222           0.1        0.424658   
76   2014-05-25 22:20:45    28.654167           0.1        0.330912   
77   2014-05-29 02:

In [8]:
fire_locations.to_csv('../output/suppression_stats.csv', index=False,
                      columns=['UniqueFireIdentifier', 'FireDiscoveryDateTime', 'HoursBurned',
                               'IncidentSize', 'NormalizedTime', 'NormalizedAcreage', 'SuppressionResult'])

filtered_fire_locations.to_csv('../output/suppression_stats_filt.csv', index=False,
                               columns=['UniqueFireIdentifier', 'FireDiscoveryDateTime', 'HoursBurned',
                                        'IncidentSize', 'NormalizedTime', 'NormalizedAcreage', 'SuppressionResult'])